# Product Demand Forecast

In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib as plt

## Explore data

### Read input data

In [117]:
orig_demand_df = pd.read_csv('Historical Product Demand.csv')
print('Original: ' + str(orig_demand_df.shape))
orig_demand_df.head(4)

Original: (1048575, 5)


,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500


### Data preprocessing

In [119]:
# convert demand to int
demand_df = orig_demand_df.copy()
demand_df.Order_Demand = orig_demand_df.Order_Demand.str.replace('\(|\)', '').astype(int)
# convert to datetime
demand_df.Date = pd.to_datetime(demand_df.Date)
# remove NA
demand_df.dropna(inplace=True)
print('After   : ' + str(demand_df.shape))

After   : (1037336, 5)


A little summary for our data

In [143]:
print('Date range: ({}, {})'.format(demand_df.Date.min(), demand_df.Date.max()))
demand_df.drop('Date', axis=1).describe(include='all').iloc[:4,:]

Date range: (2011-01-08 00:00:00, 2017-01-09 00:00:00)


,Product_Code,Warehouse,Product_Category,Order_Demand
count,1037336,1037336,1037336,1037336.0
unique,2160,4,33,NaN
top,Product_1359,Whse_J,Category_019,NaN
freq,16936,764447,470266,NaN


### Generic plots